# What's a Database

<img src='../images/Screen Shot 2019-04-01 at 11.33.07 AM.png' />

<img src='../images/Screen Shot 2019-04-01 at 11.39.04 AM.png' />

<img src='../images/Screen Shot 2019-04-01 at 11.39.23 AM.png' />

In [1]:
import sqlite3
import numpy as np
import pandas as pd

diet_df = pd.read_csv("../data/animal_food.csv")
animal_df = pd.read_csv("../data/animals.csv", 
                        parse_dates=["birthdate"],
                        infer_datetime_format=True)
conn = sqlite3.connect("../data/animal_food.db")

cur = conn.cursor()

cur.execute('drop table if exists diet')
diet_df.to_sql("diet", 
               conn,
               index=False)

cur.execute('drop table if exists animals')
animal_df.to_sql("animals", 
                 conn, 
                 dtype={'name': 'text', 'species': 'text', 'birthdate': 'date'},
                 index=False)


In [2]:
animal_df.describe()

,name,species,birthdate
count,8,8,8
unique,7,5,8
top,Max,gorilla,2010-07-23 00:00:00
freq,2,3,1
first,NaN,NaN,1989-09-15 00:00:00
last,NaN,NaN,2012-02-20 00:00:00


In [3]:
animal_df.head()

,name,species,birthdate
0,Max,gorilla,2001-04-13
1,Sue,gorilla,1998-06-12
2,Max,moose,2012-02-20
3,Alison,llama,1997-11-24
4,George,gorilla,2011-01-09


In [4]:
cur = conn.cursor()
query = "select * from diet"
pd.read_sql_query(query, conn)


,species,food
0,llama,plants
1,brown bear,fish
2,brown bear,meat
3,brown bear,plants
4,orangutan,plants
5,orangutan,insects


In [5]:
cur = conn.cursor()
query = "select * from animals"
pd.read_sql_query(query, conn)


,name,species,birthdate
0,Max,gorilla,2001-04-13 00:00:00
1,Sue,gorilla,1998-06-12 00:00:00
2,Max,moose,2012-02-20 00:00:00
3,Alison,llama,1997-11-24 00:00:00
4,George,gorilla,2011-01-09 00:00:00
5,Spot,iguana,2010-07-23 00:00:00
6,Ratu,orangutan,1989-09-15 00:00:00
7,Eli,llama,2002-02-22 00:00:00


In [6]:
cur = conn.cursor()
query = """
select animals.name, animals.species, diet.food
from animals join diet
on animals.species = diet.species
where food = 'plants'
"""
pd.read_sql_query(query, conn)


,name,species,food
0,Alison,llama,plants
1,Eli,llama,plants
2,Ratu,orangutan,plants


In [13]:
query = """
select animals.species, count(diet.species) as num, diet.food
from animals left join diet
on animals.species = diet.species
group by animals.species, diet.food
"""

pd.read_sql_query(query, conn)

,species,num,food
0,gorilla,0,None
1,iguana,0,None
2,llama,2,plants
3,moose,0,None
4,orangutan,1,insects
5,orangutan,1,plants


In [14]:
query = """
select animals.species, count(animals.species) as num, diet.food
from animals left join diet
on animals.species = diet.species
group by animals.species, diet.food
"""

pd.read_sql_query(query, conn)

,species,num,food
0,gorilla,3,None
1,iguana,1,None
2,llama,2,plants
3,moose,1,None
4,orangutan,1,insects
5,orangutan,1,plants


In [58]:
cur = conn.cursor()
query = "select name from sqlite_master where type='table'"
pd.read_sql_query(query, conn)


,name
0,diet
1,animals


In [64]:
cur = conn.cursor()
query = "pragma table_info(animals)"
pd.read_sql_query(query, conn)


,cid,name,type,notnull,dflt_value,pk
0,0,name,text,0,None,0
1,1,species,text,0,None,0
2,2,birthdate,date,0,None,0
